# Hosting your model on Google Cloud ML

https://cloud.google.com/ml-engine/docs/tensorflow/deploying-models

This requires installation of the Google Cloud SDK
* https://cloud.google.com/sdk/install
  * https://cloud.google.com/sdk/docs/downloads-interactive


In [3]:
!ls -l tf/1

total 616
-rw-r--r-- 1 olive 197609 625641 Sep 14 09:35 saved_model.pb
drwxr-xr-x 1 olive 197609      0 Sep 14 09:35 variables


## First we need to create a bucket on the Goolge Cloud and upload our model to it

https://cloud.google.com/storage/docs/creating-buckets#storage-create-bucket-gsutil


In [17]:
!gsutil mb gs://manning_bucket
!gsutil cp -R tf/1 gs://manning_bucket

'gsutil' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!gcloud ml-engine models create "manning_insurance_1"
!gcloud ml-engine versions create "v1" --model "manning_insurance_1" --origin gs://manning_bucket/1    
!gcloud ml-engine versions describe "v1" --model "manning_insurance_1"

## Try it out on a few samples

In [19]:
# one of each category
!cat sample_insurance.json

{"inputs": [ 160,  18,  100]}
{"inputs": [ 100,  47,  10]}
{"inputs": [ 90,  20,  20]}


In [4]:
# 0: red
# 1: green
# 2: yellow

# https://cloud.google.com/ml-engine/docs/tensorflow/prediction-overview#getting_predictions
!gcloud ml-engine predict --model "manning_insurance_1" --version "v1" --json-instances ./sample_insurance.json

# SCORES
# [0.8658562898635864, 7.318668918511809e-14, 0.13414366543293]
# [0.002760800765827298, 0.8720880746841431, 0.12515118718147278]
# [5.452934419736266e-05, 0.005952719133347273, 0.9939927458763123]

ERROR: (gcloud.ml-engine.predict) HTTP request failed. Response: {
  "error": {
    "code": 403,
    "message": "Cloud Machine Learning Engine has not been used in project 564044401623 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=564044401623 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.",
    "status": "PERMISSION_DENIED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "description": "Google developers console API activation",
            "url": "https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=564044401623"
          }
        ]
      }
    ]
  }
}



## You can also access our deployed model programmatically

https://cloud.google.com/ml-engine/docs/tensorflow/online-predict

In [3]:
!pip install google-api-python-client

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
thinc 6.11.2 requires hypothesis<3,>=2, which is not installed.
thinc 6.11.2 requires msgpack-python==0.5.4, which is not installed.
thinc 6.11.2 has requirement msgpack-numpy==0.4.1, but you'll have msgpack-numpy 0.4.3 which is incompatible.
tensorflow-tensorboard 1.5.1 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
tensorflow-tensorboard 1.5.1 has requirement html5lib==0.9999999, but you'll have html5lib 1.0.1 which is incompatible.
spacy 2.0.11 has requirement regex==2017.4.5, but you'll have regex 2018.6.21 which is incompatible.
spacy 2.0.11 has requirement thinc<6.11.0,>=6.10.1, but you'll have thinc 6.11.2 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [24]:
!pip install tensorflow-serving-api

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
thinc 6.11.2 requires hypothesis<3,>=2, which is not installed.
thinc 6.11.2 requires msgpack-python==0.5.4, which is not installed.
thinc 6.11.2 has requirement msgpack-numpy==0.4.1, but you'll have msgpack-numpy 0.4.3 which is incompatible.
tensorflow-tensorboard 1.5.1 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
tensorflow-tensorboard 1.5.1 has requirement html5lib==0.9999999, but you'll have html5lib 1.0.1 which is incompatible.
spacy 2.0.11 has requirement regex==2017.4.5, but you'll have regex 2018.6.21 which is incompatible.
spacy 2.0.11 has requirement thinc<6.11.0,>=6.10.1, but you'll have thinc 6.11.2 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
# https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/ml_engine/online_prediction/predict.py
# https://cloud.google.com/ml-engine/docs/tensorflow/online-predict

import googleapiclient.discovery

def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [2]:
instances = [{"inputs": [ 160,  18,  100]}, {"inputs": [ 100,  47,  10]}, {"inputs": [ 90,  20,  20]}]
predict_json("sandboxolli", "manning_insurance_1", instances=instances)

[{'scores': [0.8658562898635864, 7.318668918511809e-14, 0.13414366543293]},
 {'scores': [0.002760800765827298, 0.8720880746841431, 0.12515118718147278]},
 {'scores': [5.452934419736266e-05, 0.005952719133347273, 0.9939927458763123]}]